<a href="https://colab.research.google.com/github/leshow123/An-Abstractive-Text-Summarizer-For-Patent-Documents/blob/main/Project_Patent_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U -q PyDrive
!pip install -q -U tensor2tensor

     |████████████████████████████████| 993kB 3.5MB/s 
     |████████████████████████████████| 1.4MB 2.1MB/s 
     |████████████████████████████████| 143kB 64.7MB/s 
     |████████████████████████████████| 686kB 64.4MB/s 
     |████████████████████████████████| 296kB 67.8MB/s 


# Utilities

In [ ]:
def dataset_update(dataset_in_cloud='/content/drive/My Drive/sample.csv'):
  """ Use this to extract 'unclean' data from IamIPAB's raw dump """
  import glob
  import pandas as pd
  
  #TODO : Handle file not found scenario etc.
  
  file_path = glob.glob(dataset_in_cloud, recursive=True).pop(0)
  print(file_path)
  df = pd.read_csv(file_path,sep=';',encoding='latin-1', error_bad_lines=False,
                   warn_bad_lines=False, header=0)
  number_of_columns = len(list(df.columns))
  datapoints = int(df.size/number_of_columns)
  
  df[:datapoints].to_csv('/content/drive/My Drive/Thesis_at_IamIP/data_dir/sample_{}_dp.csv'.format(datapoints), sep=';')
  print('INFO:Finished writing sample_{}_dp.csv'.format(datapoints) )

In [ ]:
def touch(path):
  
  success = True
  try:
    with open(path, 'a') as en_dataset:
      en_dataset.write('Title'+'@'+'Abstract'+'@'+'Claims'+'@'+'Description'+'\n')
  except:
    success = False
  
  return success

In [ ]:
def extract_english(*args):
  
  # Reference 'most common by freq': https://www.wordfrequency.info/free.asp?s=y
  # Note: Took out 'in' as its prevalent in German too;
  #       Took out 'a' too
  
  ptrn_assert_en = r'\b\s*(the|be|and|of|to|have|it|that)\s+'
  
  value = str()
  length = len(args)
  items = length-2
  toggle_bit = [False]*items
  limit = length-1
  datapoint = args[limit-1]
  idx = 0
  
  for n,arg in enumerate(args):
    
    if n == items:
      break
      
    idx = args[limit].loc[datapoint,arg].find('en: ')
   
    if idx == -1:
      value = ''
      toggle_bit[n] = True
    else: # en: exists
      # is it truely english despite its label en: ? See datapoint 32 as e.g.
      
      found = [m for m in map(str.lower, 
               re.findall(ptrn_assert_en, args[limit].loc[datapoint,arg],
               flags=re.IGNORECASE))]
      
      if len(found)!=0:
        
        itera = re.finditer(r"\.\s+[a-z][a-z]:", args[limit].loc[datapoint,arg])
        indices = [m.start(0) for m in itera]
        for indx in indices:
          if idx < indx: 
            value = args[limit].loc[datapoint,arg][idx:indx] 
            # just run op once
            break

        else:    
          value = args[limit].loc[datapoint,arg][idx:]
      else:
        # not in english despite label
        value = ''
        toggle_bit[n] = True
  
  return toggle_bit, value

In [ ]:
def write_to_file(data, en_dataset_path):
  
  success = True
  try: # data => <str, str, list, list>
    with open(en_dataset_path, 'a') as en_dataset:
        en_dataset.write(data[0] + '@' + data[1])
        
        en_dataset.write('@')
        
        for itr in range(len(data[2])):
          en_dataset.write(data[2][itr] + ' ')
        
        en_dataset.write('@')
        
        for its in range(len(data[3])):  
          en_dataset.write(data[3][its] + ' ')
        
        en_dataset.write('\n')
  except:
    success = False 
  
  data.clear()
  return success

In [ ]:
def write_vocab_to_file(vocab_list, dest_file):
  
  success = True
  try:
    with open(dest_file, 'w') as vocab:
        # Write RESERVED TOKENS expected by Tensor2Tensor
        vocab.write('<pad>')
        vocab.write('\n')
        vocab.write('<EOS>')
        # other writes follow...
        for token in vocab_list:
          vocab.write('\n' + token)
  except:
    success = False 
  
  vocab_list.clear()
  return success

In [ ]:
def data_generator(en_dataset_path, split):
    
    df = pd.read_csv(en_dataset_path,sep='@',error_bad_lines=False, 
                     warn_bad_lines=False, header=0)   
    number_of_columns = len(list(df.columns))
    for i in range(int(df.size/number_of_columns)): #size = number of datapoints x (number of columns)
      abstract = str(df.loc[i,'Abstract'])   
      claims = str(df.loc[i,'Claims'])
      split_str = u' <split> ' if split else " "
      yield abstract + split_str + claims

In [ ]:
def token_text_encoder(vocab_list, dest_filename):
    """ Takes in UNIQUE vocab for encoding by TokenTextEncoder class """
    
    from tensor2tensor.data_generators import text_encoder
   
    vocab_list.append("UNK")
    try:
      te = text_encoder.TokenTextEncoder(vocab_filename=None,
                 reverse=False,
                 vocab_list=vocab_list,
                 replace_oov="UNK",   #Note: implementing backoff
                 num_reserved_ids=2) # <bos> (or <pad>) and <eos>
    except:
      pass
    
    try:
      te.store_to_file(dest_filename)
    except:
      pass

In [ ]:
def preprocess_data(find_dataset, en_dataset_path):

  import time
  import progressbar
  
  #Patterns
  ptrn_title_en = re.compile(r'en:.*')
  ptrn_abstract_en = re.compile(r'en:\s*.*')
  neg_ptrn_lead_xter = re.compile(r'en:\s*')
  neg_ptrn_slash_es = re.compile(r'\b\s+')
  sentences = re.compile(r'\b.*\.')
  extraneous = re.compile(r'(CLAIMS1|CLAIMS1.|claims1|claims1.)')
  
  csv_output_buffer = []
  en_title = []
  en_abstract = ['']
  en_claims = ''
  en_description = ''
  lines_written = 0
  
  df = pd.read_csv(find_dataset,sep=';', error_bad_lines=False, 
                   warn_bad_lines=False, header=0)  
  number_of_columns = len(list(df.columns))
  datapoints = int(df.size/number_of_columns)

  # Note: The following still falls through the crack:
  #       1. Scenario where another language is embedded as lines along with
  #          English. Example, datapoint 30
  #       2. Abstracts are often too short to test for most common english words
  
  bar = progressbar.ProgressBar()
  for i in bar(range(datapoints)):
    
    # Note : If all of Abstract, Claims or Description is not in English or
    #        dont exist we will skip that datapoint 
    # Note: Some datapoints though tagged 'en:', aint in english; refer dp 99

    toggle_bit_on_claims = False
    toggle_bit_on_abstract = False
    toggle_bit_on_desc = False
    
    try:
      en_title = re.findall(ptrn_title_en, df.loc[i,'Title'])
    except:
      continue # in any event that 'en:' pattern  is not found 
    
    # Handle multiple english titles
    if len(en_title) > 1:
      for k in range(len(en_title)-1):
        en_title.pop(0)
    # Handle title-less docs (even after en: pattern found) with a placeholder 
    if len(en_title) == 0:
        en_title = ['LOREM_IPSUM']
        
    # Note: Some datapoints dont have either of abstracts, claims or description
    try: 
      en_abstract = re.findall(ptrn_abstract_en, df.loc[i,'Abstract']) 

      # Handle multiple english abstracts e.g in datapoint 14 
      if len(en_abstract) > 1:
        for j in range(len(en_abstract)-1):
          en_abstract.pop(0)

      if len(en_abstract) == 0:
        en_abstract = ['']
        toggle_bit_on_abstract = True

    except:  
      # Also toggle-on when it dont exist or for some other unknown
      toggle_bit_on_abstract = True
      # Note: Template <COL1, COL2,...COLN, i, dataframe> for fxn extract_english
    try:
      toggler, en_claims = extract_english('Claims',i,df)
      toggle_bit_on_claims = toggler[0]
    except:
      toggle_bit_on_claims = True
    try:  
      toggler, en_description = extract_english('Description',i,df)
      toggle_bit_on_desc = toggler[0]
    except:
      toggle_bit_on_desc = True

    # Test for fidelity: No point in retaining datapoint if one of the following
    # is in another language.
    if toggle_bit_on_claims and toggle_bit_on_desc:
      continue

    ## Further data processing
    if len(en_title) != 0:
      en_title[0] = neg_ptrn_lead_xter.sub('', en_title[0])
    if len(en_abstract) != 0:
      en_abstract[0] = neg_ptrn_lead_xter.sub('', en_abstract[0])
    if en_claims is not '':
      en_claims = neg_ptrn_lead_xter.sub('', en_claims)
    if en_description is not '':
      en_description = neg_ptrn_lead_xter.sub('', en_description)
    
    # Take out hidden spaces, tabs etc and break-off into sentences
    copy_en_desc = neg_ptrn_slash_es.sub(' ', en_description)
    copy2_en_desc = [''.join(itm) for itm in re.findall(sentences, copy_en_desc)]   
    
    copy_en_claims = neg_ptrn_slash_es.sub(' ', en_claims)  
    copy1_en_claims = extraneous.sub(' ', copy_en_claims)
    copy2_en_claims = [''.join(itm) for itm in re.findall(sentences, copy1_en_claims)]
    
    if len(copy2_en_desc) == 0 and len(copy2_en_claims) == 0:
      data = [en_title[0], en_abstract[0], [' '], [' ']]
    elif len(copy2_en_claims) == 0 and len(copy2_en_desc) != 0:
      data = [en_title[0], en_abstract[0], [' '], copy2_en_desc[:]]
    elif len(copy2_en_desc) == 0 and len(copy2_en_claims) != 0:
      data = [en_title[0], en_abstract[0], copy2_en_claims[:], [' ']]
    else:
      data = [en_title[0], en_abstract[0], copy2_en_claims[:], copy2_en_desc[:]]
    
    success = write_to_file(data, en_dataset_path)
    if success:
      lines_written = lines_written + 1
    else:
      exit("Failed to write file: {}; Program exiting.".format(en_dataset_path))
    
  return lines_written

# Modules

** \__init__  ** : Indicate to T2T there's a custom problem to load

In [ ]:
%%writefile __init__.py
from . import registrar

Writing __init__.py


**registrar** - register summarization problem instance with tensor2tensor

In [ ]:
%%writefile registrar.py

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from tensor2tensor.data_generators import generator_utils
from tensor2tensor.data_generators import problem
from tensor2tensor.data_generators import text_encoder
from tensor2tensor.data_generators import text_problems
# from tensor2tensor.data_generators import wiki_lm
from tensor2tensor.utils import registry
import os

def splitter(document):
  
    split_str = u' <split> '
    split_str_len = len(split_str)
    split_pos = document.find(split_str)
    return document[:split_pos], document[split_pos + split_str_len:]  # abstract, claims

def example_generator(split):
    
    import pandas as pd
    
    # TODO : pass the dataset location in by reading from a file locally prior
    #        written to by main()
    
    en_dataset_path = '/content/drive/My Drive/Thesis_at_IamIP/data_dir/sample_6978_dp_clean.csv'
    #en_dataset_path = '/content/drive/My Drive/Thesis_at_IamIP/data_dir/sample_1944_dp_clean.csv'
    
    df = pd.read_csv(en_dataset_path,sep='@',error_bad_lines=False, 
                     warn_bad_lines=False, header=0)
    number_of_columns = len(list(df.columns))
    for i in range(int(df.size/number_of_columns)): #size = number of datapoints x (number of columns)
      abstract = str(df.loc[i,'Abstract'])   
      claims = str(df.loc[i,'Claims'])
      split_str = u' <split> ' if split else " "
      yield abstract + split_str + claims

@registry.register_problem('summarize_patent')
class SummarizePatent(text_problems.Text2TextProblem):

  def write_raw_text_to_files(self, dataset_split, tmp_dir):

    def write_to_file(tmp_dir, filename):
      """Write text to files."""
      with open(os.path.join(tmp_dir, filename + ".source"), "w") as fstory:
        with open(os.path.join(tmp_dir, filename + ".target"), "w") as fsummary:
          for document in example_generator(True): 
            abstract, claims = splitter(document)
            fstory.write(claims + "\n")
            fsummary.write(abstract + "\n")
   
    if dataset_split == problem.DatasetSplit.TRAIN:
      filename = "patentsumm.train"
    elif dataset_split == problem.DatasetSplit.EVAL:
      filename = "patentsumm.dev"
    else:
      filename = "patentsumm.test"
    
    write_to_file(tmp_dir, filename)
 
  # Note: The following functions are the least 2 that must be overriden
  #       of text_problems. Refer /data_generators/text_problems.py 
  
  def is_generate_per_split(self):              
    return False # do auto split and base it on ratio defined by dataset_split
  
  def generate_samples(self, data_dir, tmp_dir, dataset_split):
    
    del data_dir
    self.write_raw_text_to_files(dataset_split, tmp_dir)
    split = True
    for document in example_generator(split):
      abstract, claims = splitter(document)
      yield {"inputs": claims, "targets": abstract}
  
  ## Other overrides
  
  def generate_text_for_vocab(self, data_dir, tmp_dir):
    del data_dir
    del tmp_dir
    split = False
    return example_generator(split)
  
  # Modifiable properties
  
  @property
  def vocab_type(self):
    return text_problems.VocabType.TOKEN   # refer text_problems.py
  
  
  @property
  def dataset_splits(self):
    """ Splits of data to produce and number of output shards for each. """
    return [{
        "split": problem.DatasetSplit.TRAIN,
        "shards": 100,
    }, {
        "split": problem.DatasetSplit.EVAL,
        "shards": 10,
    }, {
        "split": problem.DatasetSplit.TEST,
        "shards": 10,
    }]
  
  @property
  def oov_token(self):
    """Out of vocabulary token. Only for VocabType.TOKEN."""
    return "UNK"

Writing registrar.py


**data_processing**: Preprocess the data and build vocab

In [ ]:
%%writefile data_processing.py

import os

def processor(find_dataset, en_dataset_path):
  
  if not os.path.exists(en_dataset_path):    

    # Touch en_dataset
    success = touch(en_dataset_path)
    if not success:
      exit("Problem creating file: {}; Program exiting.".format(en_dataset_path))

    print("File created: {}".format(en_dataset_path), '\n')

    lines_written = preprocess_data(find_dataset, en_dataset_path)

    print('Total Number of Lines Written to {} : '.format(en_dataset_path), 
          lines_written)
  else:
    print("File already exists: {}".format(en_dataset_path), '\n')
  
  return

Writing data_processing.py


**vocabulary**: Build vocabulary locally from clean english dataset

In [ ]:
%%writefile vocabulary.py

import string
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

END_TOKENS = [u'?', u'.', u'!', u',', u'\'', u'"',u"`", u")"]

# TODO: Import utility.py to gain acess to utility functions used here
#from . import utility 

def create_vocab(en_dataset_path, save_to_dir):
  
  stop_words = set(stopwords.words('english'))
  # prep to remove punctutaions 
  table = str.maketrans('', '', string.punctuation)
  
  feedback = data_generator(en_dataset_path, False)
  
  #TODO: remove instances of NAN during that seep into the data during pre-processing 

  #QUERY: LEMATIZATION?
  count = 0
  tokens = []
  vocab_list = []
  for item in feedback:
    for token in word_tokenize(item.lower().strip()):
      count = count + 1  
      if token not in stop_words:
        if token not in END_TOKENS and token.isalnum():  #### OR ISALPHA?
          tokens.append(token)

  depunctuated_tokens = [token.translate(table) for token in tokens]        
  # remove duplicates, if any
  tokens.clear()
  tokens = list(dict.fromkeys(depunctuated_tokens)) 
  vocab_list.extend(tokens)
  # append UNK for back-off: vocab requirement imposed by tensor2tensor
  vocab_list.append("UNK")

  # Note: Theres a format to be followed in naming vocab file for T2T problems
  success = write_vocab_to_file(vocab_list, save_to_dir + u'/vocab.summarize_patent.tokens')
  
  if success:
    print("INFO: Vocabulary successfully written to: {}; ".format(
      save_to_dir + u'/vocab.summarize_patent.tokens'))
  else:
    print("INFO: Failed to write vocabulary to: {}; ".format(
      save_to_dir + u'/vocab.summarize_patent.tokens'))
    return sys.exit('ERROR: Program exiting.')
  return

Writing vocabulary.py


**cloud_download** : Copy cloud/gdrive file(s) locally

In [ ]:
%%writefile clouddownload.py

import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

def copy_cloud_file_locally(*args):
  """ Download file(s) from cloud/drive to local_path """
  
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)
  
  local_path, filenames = args
  
  if filenames is None:
    return "Error: You must supply a filename."
  
  # choose a local (colab) directory to store the data.
  local_download_path = os.path.expanduser(local_path)
  
  for filename in filenames:
    try:
      # TODO: Handle case where a replica of a file possibly existing in Trash...
      listed = drive.ListFile({'q':"title contains " + "'" + filename + "'", 'spaces':'drive'}).GetList()
      for file in listed:
        fname = os.path.join(local_download_path, file['title'])
        print('INFO: Downloading to {}'.format(fname))
        f_ = drive.CreateFile({'id': file['id']})
        f_.GetContentFile(fname)
    except:
      return "Error: File not found in source: cloud/drive."

Writing clouddownload.py


**\__main__**: Copy cloud/gdrive file(s) locally

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import pandas as pd
import argparse
import os
import sys
import re

tf.logging.set_verbosity(tf.logging.INFO)

#Enable TF Eager execution
tfe = tf.contrib.eager
tfe.enable_eager_execution()

Modes = tf.estimator.ModeKeys

from tensor2tensor import problems
from tensor2tensor.layers import common_layers
from tensor2tensor.utils import trainer_lib
from tensor2tensor.utils import t2t_model
from tensor2tensor.utils import registry
from tensor2tensor.utils import metrics
from tensor2tensor.data_generators import text_encoder

#from . import clouddownload 
#from . import data_processing
#from . import vocabulary
exec(compile(open('clouddownload.py', "rb").read(), 'clouddownload.py', 'exec'))
exec(compile(open('data_processing.py', "rb").read(), 'data_processing.py', 'exec'))
exec(compile(open('vocabulary.py', "rb").read(), 'vocabulary.py', 'exec'))

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

FLAGS = None
DEFAULT_FILES_DDIR = ['sample_101010_dp_clean.csv']

#ptrn_path = re.compile(r'\/.*\/')
ptrn_dp = re.compile(r'_\d*_')

def main():
  
    # TODO Handle where all flags empty
   
  t2t_usr_dir = FLAGS.t2t_usr_dir  
  data_dir = FLAGS.data_dir
  tmp_dir = FLAGS.tmp_dir
  train_dir = FLAGS.train_dir
  problem = FLAGS.problem
  more_cloud_files_to_download = FLAGS.download_cloud_files
  
    # TODO: Consider using os.makedir() ...
  if not os.path.exists(data_dir):
    tf.gfile.MakeDirs(data_dir)
  if not os.path.exists(tmp_dir):
    tf.gfile.MakeDirs(tmp_dir)
  if not os.path.exists(train_dir):
    tf.gfile.MakeDirs(train_dir)
  
  # data-(pre)processing call
  
    #TODO : Read the raw dataset location in cloud in with flag
  find_dataset = u'/content/drive/My Drive/Thesis_at_IamIP/data_dir/sample_6978_dp.csv'
  #find_dataset = u'/content/drive/My Drive/Thesis_at_IamIP/data_dir/sample_1944_dp.csv'
  
  dp = re.findall(ptrn_dp, find_dataset).pop(0)
  dp = int(dp.lstrip('_').rstrip('_'))
  
  data_dir_path_in_cloud, _ = os.path.split(find_dataset)
  en_dataset_path = data_dir_path_in_cloud + u'/' + 'sample_{}_dp_clean.csv'.format(dp)
  processor(find_dataset, en_dataset_path)
  
  # create vocab (locally) call
  create_vocab(en_dataset_path, data_dir)
    
  # Add the clean english dataset to list of files to download from cloud
  add_file = 'sample_{}_dp_clean.csv'.format(dp)
  DEFAULT_FILES_DDIR.append(add_file)
  files_to_dload_to_ddir = DEFAULT_FILES_DDIR
  
  if not FLAGS.download_cloud_files is None:
    # Assuming they're in .../data_dir in CLOUD
    add_default_files = [str.strip(fl) for fl in more_cloud_files_to_download.split(',')]    
    files_to_dload_to_ddir.extend(add_default_files)
    copy_cloud_file_locally(data_dir, files_to_dload_to_ddir)
  
  # ...where there's no additional files to download locally to --data_dir
  copy_cloud_file_locally(data_dir, files_to_dload_to_ddir)
  
  #TODO: Remove eventually; in the mean time, simulating CMD-line
  #!t2t-datagen --t2t_usr_dir='/content' --data_dir='/content/data_dir' --tmp_dir='/content/tmp_dir' --problem=summarize_patent
  !t2t-datagen --t2t_usr_dir='/content' --data_dir='/content/data_dir' --tmp_dir='/content/tmp_dir' --problem=summarize_patent
  
  # TODO: code to commence training should come here
  !t2t-trainer --data_dir='/content/data_dir' --problem=summarize_patent \
    --model=transformer --hparams_set=transformer_base_single_gpu \
    --output_dir='/content/train_dir' \
    --hparams='batch_size=1024' \
    --t2t_usr_dir='/content' \
    --train_steps=140000
  
  # TODO: code to infer should come here; perhaps a filename is passed to main
  # where data can be read from
  
  ## Decode; NOTE: Toggle --decode_from_file to use eval/test set rather
  #--decode_from_file='/content/data_dir/decode_data_3560.txt' \
  
  !t2t-decoder --data_dir='/content/data_dir' --problem=summarize_patent \
  --model=transformer --hparams_set=transformer_base_single_gpu \
  --output_dir='/content/train_dir' --decode_hparams="beam_size=4,alpha=0.6" \
  --t2t_usr_dir='/content' \
  --decode_to_file=summarized_decoded.txt
  
  
if __name__ == '__main__':
    
  #main()
  
  parser = argparse.ArgumentParser()
  parser.add_argument(
      '--t2t_usr_dir',
      type=str,
      default='/content',
      help='Location of __init__ menat to import user components.')
  parser.add_argument(
      '--data_dir',
      type=str,
      default='/content/data_dir',
      help="""\
      Where to locate data files and vocabulary file.
      """)
  parser.add_argument(
      '--tmp_dir',
      type=str,
      default='/content/tmp_dir',                                                                      
      help="""\
      Where to locate any other related files e.g. *.source and *.target files.
      """)
  parser.add_argument(
      '--train_dir',
      type=str,
      default='/content/train_dir',                                                                      
      help="""\
      Output of training process will be put here.
      """)
  parser.add_argument(
      '--download_cloud_files',
      type=str,
      default=None,                                                                      
      help="""\
      Comma seperated list of filenames to download form cloud data_dir folder.
      """)
  
  parser.add_argument(
      '--problem',
      type=str,
      default='summarize_patent',
      help="""\
      Snake-cased name of the problem definition.
      """)
   
  FLAGS, unparsed = parser.parse_known_args()
  main()
  #tf.app.run(main=main, argv=[sys.argv[0]] + unparsed)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:Entry Point [tensor2tensor.envs.tic_tac_toe_env:TicTacToeEnv] registered with id [T2TEnv-TicTacToeEnv-v0]
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Faut

# Data Generation and Training

In [ ]:
# Generate Data
"""
!t2t-datagen --t2t_usr_dir='/content' --data_dir='/content/data_dir' --tmp_dir='/content/tmp_dir' --problem=summarize_patent
"""

In [ ]:
# Training

#--hparams_set=transformer_base_single_gpu
#--hparams_set=transformer_tpu
#--hparams_set=transformer_prepend
"""
!t2t-trainer --data_dir='/content/data_dir' --problem=summarize_patent \
    --model=transformer --hparams_set=transformer_base_single_gpu \
    --output_dir='/content/train_dir' \
    --hparams='batch_size=1024' \
    --t2t_usr_dir='/content' \
    --train_steps=140000
"""

# Decoding (Inference)

In [ ]:
# Take datapoints from an unseen dataset
"""
decode_file = u'/content/data_dir/sample_101010_dp_clean.csv'

df = pd.read_csv(decode_file,sep='@',error_bad_lines=False, 
                     warn_bad_lines=False, header=0)   

number_of_columns = len(list(df.columns))
dp_test_data = int(df.size/number_of_columns)   #size = number of datapoints x (number of columns)
start= dp_test_data - 2                         # ==> 1---1944--------------dp_test_data{3570} 

for i in range(start, dp_test_data): 
  dest_file = u'/content/data_dir/decode_data_{}.txt'.format(i)
  abstract = str(df.loc[i,'Abstract'])   
  claims = str(df.loc[i,'Claims'])
  data = abstract + " " + claims  
  with open(dest_file, 'w') as testdata:
    testdata.write(data)

"""